https://python.langchain.com/docs/use_cases/sql/quickstart/

### **Test the sqldb**

In [2]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

**Connecting to the sqldb**

In [7]:
db_path = str(here("data")) + "/sqldb.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [8]:
db

In [ ]:
# validate the connection to the vectordb
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

### **Test the access to the environment variables**

In [3]:
from dotenv import load_dotenv
import os

here(".env")
print("Environment variables are loaded:", load_dotenv(dotenv_path=here(".env")))
print("test by reading a variable:", os.getenv("OPENAI_API_VERSION"))
print("test by reading a variable:", os.getenv("OPENAI_API_BASE"))
print("test by reading a variable:", os.getenv("OPENAI_API_KEY"))
print("test by reading a variable:", os.getenv("OPENAI_API_TYPE"))

Environment variables are loaded: True
test by reading a variable: 2024-05-01-preview
test by reading a variable: https://cogopenaiscgjwdllmchat1.openai.azure.com/
test by reading a variable: dc1475322fbf4c07a1469f57242f14b5
test by reading a variable: azure


### **Test your GPT model**

In [ ]:
from openai import AzureOpenAI

messages = [{"role": "system", "content": str("You are a helpful assistant")}, {"role": "user", "content": str("hello")}]
client = AzureOpenAI(
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY"),
)

response = client.chat.completions.create(model="gpt-35-turbo", messages=messages)
print(response.choices[0].message.content)

### **1. SQL query chain**

In [28]:
# Load the LLM
from langchain_community.chat_models import AzureChatOpenAI

model_name = "gpt-35-turbo"
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]
llm = AzureChatOpenAI(openai_api_version=os.getenv("OPENAI_API_VERSION"), azure_deployment=model_name, model_name=model_name, temperature=0.0)

In [29]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})

Execute the query to make sure it’s valid

In [26]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [36]:
from sqlalchemy import create_engine, inspect

# Create an engine that connects to the test.db SQLite database
engine = create_engine(f"sqlite:///{db_path}")

# Connect to the database
connection = engine.connect()

# Create an inspector object
inspector = inspect(engine)

# Retrieve the names of all the tables in the database
table_names = inspector.get_table_names()
db_schema_txt = "Tables:" + ",".join(table_names)

# Loop over each table to get detailed information like schema, columns, etc.
for table_name in table_names:
    db_schema_txt += f"Information for table: {table_name}"

    # Get the schema of the table (for SQLite, schema is often None)
    db_schema_txt += f"Schema: {inspector.get_schema_names()}"

    # Get the columns and their attributes for each table
    columns = inspector.get_columns(table_name)
    for column in columns:
        db_schema_txt += f"Column: {column['name']} Type: {column['type']}"

    # Additionally, you can use get_pk_constraint and get_foreign_keys
    # methods to retrieve information about primary and foreign keys respectively
    pk_constraint = inspector.get_pk_constraint(table_name)
    db_schema_txt += f"Primary Key Constraint: {pk_constraint}"

    foreign_keys = inspector.get_foreign_keys(table_name)
    db_schema_txt += f"Foreign Keys: {foreign_keys}"


db_schema_txt
# Do not forget to close the connection when done
connection.close()

pond_sql_tool_prompt = """
You are a SQLite expert.
You will only respond to the Query commands from the User Question without explaining anything.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}
""".format(
    table_info=db_schema_txt,
    input="ใบแจ้งหนี้ทั้งหมดรวมเป็นเงินกี่บาท",
)

print(pond_sql_tool_prompt)


You are a SQLite expert.
You will only respond to the Query commands from the User Question without explaining anything.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
Tables:Album,Artist,Customer,Employee,Genre,Invoice,InvoiceLine,MediaType,Playlist,PlaylistTrack,TrackInformation for table: AlbumSchema: ['main']Column: AlbumId Type: INTEGERColumn: Title Type: NVARCHAR(160)Column: ArtistId Type: INTEGERPrimary Key Constraint: {'constrained_columns': ['AlbumId'], 'name': None}Foreign Keys: [{'name': None, 'constrained_columns': ['ArtistId'], 'referred_schema': None, 'referred_table': 'Artist', 'referred_columns': ['ArtistId'], 'options': {}}]Information for table: ArtistSchema: ['main']Column: ArtistId Type: INTEGERColumn: Name Type: NVARCHAR(120)Primary Key Constraint: {'constrained_columns': ['ArtistId'], 'name': None}Foreign Keys: []Information for table: Custo

In [39]:
response = llm.invoke(pond_sql_tool_prompt)
response.content

'SQLQuery: SELECT SUM(Total) AS TotalAmount FROM Invoice\nSQLResult: TotalAmount\nAnswer: TotalAmount in Baht'

### **Add QuerySQLDataBaseTool to the chain**
Execute SQL query

**This is the most dangerous part of creating a SQL chain.** Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution (see below).

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [51]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "How many employees are there"})

'[(8,)]'

### **Answer the question in a user friendly manner**

In [58]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)


answer = answer_prompt | llm | StrOutputParser()
chain = RunnablePassthrough.assign(query=write_query)
chain = chain.assign(result=(itemgetter("query") | execute_query)) | answer
print(chain.invoke({"question": "ในใบ invoice รวมทั้งหมดกี่ $"}))

The total amount in the invoice is $2328.6.


### **2. Agents**

Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

- It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
- It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- It can answer questions that require multiple dependent queries.
- It will save tokens by only considering the schema from relevant tables.

To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions
- …

In [59]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [60]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Invoice,Customer'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	São José 

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'The total sales per country are as follows:\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n6. United Kingdom: $112.86\n7. Czech Republic: $90.24\n8. Portugal: $77.24\n9. India: $75.26\n10. Chile: $46.62\n\nThe country whose customers spent the most is the USA, with a total sales amount of $523.06.'}

In [61]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'PlaylistTrack'}`



CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/The `PlaylistTrack` table has two columns: `PlaylistId` and `TrackId`. It represents a many-to-many relationship between playlists and tracks, with each row indicating that a specific track is part of a specific playlist. The table has a composite primary key consisting of both `PlaylistId` and `TrackId`. Additionally, there are foreign key constraints on both `PlaylistId` and `TrackId`, referenc

{'input': 'Describe the playlisttrack table',
 'output': 'The `PlaylistTrack` table has two columns: `PlaylistId` and `TrackId`. It represents a many-to-many relationship between playlists and tracks, with each row indicating that a specific track is part of a specific playlist. The table has a composite primary key consisting of both `PlaylistId` and `TrackId`. Additionally, there are foreign key constraints on both `PlaylistId` and `TrackId`, referencing the `Playlist` and `Track` tables respectively. Here are a few sample rows from the `PlaylistTrack` table:\n\n| PlaylistId | TrackId |\n|------------|---------|\n| 1          | 3402    |\n| 1          | 3389    |\n| 1          | 3390    |\n\nIf you have any specific questions about this table, feel free to ask!'}